In [1]:
%cd "C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag"

C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag


Extracting text from Docs

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from src.backend.textextractor import DocumentLoader

In [3]:
texts = DocumentLoader(path=r"src\docs").load()

Loading all files from folder: src\docs
 → Loading: src\docs\doc1.pdf
Loading file: src\docs\doc1.pdf
 → Loading: src\docs\doc2.pdf
Loading file: src\docs\doc2.pdf

✅ Total documents loaded: 60


In [4]:
texts[59]

Document(metadata={'producer': 'Acrobat Distiller 6.0 for Windows', 'creator': 'Elsevier', 'creationdate': '2008-12-01T13:19:14+05:30', 'source': 'src\\docs\\doc2.pdf', 'file_path': 'src\\docs\\doc2.pdf', 'total_pages': 38, 'format': 'PDF 1.4', 'title': 'Essentials of Cardiology', 'author': 'Timothy C. Slesnick; Ralph Gertler; Wanda C. Miller-Hance', 'subject': '', 'keywords': '', 'moddate': '2009-01-05T18:26:04+05:30', 'trapped': '', 'modDate': "D:20090105182604+05'30'", 'creationDate': "D:20081201131914+05'30'", 'page': 37}, page_content='330\nresult in a normal life expectancy and normal cardiovascular \nreserve.332 Th ey generally require no further medical or surgical \ntreatment. As such, only a few procedures fulﬁ ll these criteria: \nligation/division/occlusion of a patent ductus arteriosus and \nclosure of an isolated secundum atrial septal defect. Other inter-\nventions or surgical procedures may result in repair or “correc-\ntion,” however not necessarily in normal hemodynam

Chunking

In [5]:
from src.backend.textchunker import chunking

In [6]:
chunks = chunking(texts).chunker()

In [7]:
chunks[1].page_content

'oxygen to part of your heart muscle. \n• \nHeart valve disease: A valve in your heart isn’t working right. \n• \nHigh blood pressure: Your blood is pushing too hard against your artery walls. \n• \nHigh cholesterol: Your blood has too many fats in it. \n• \nPericarditis: Inflammation in your heart’s lining (pericardium). \nCommon signs or symptoms of heart conditions \nSymptoms of heart conditions include: \n• \nChest pain. \n• \nHeart palpitations. \n• \nDizziness. \n• \nShortness of breath. \n• \nFatigue. \n• \nSwelling in your lower body. \nCommon tests to check the health of your heart \nTests to check your heart health include:'

EMbedding and saving in Vector DB

In [8]:
from src.backend.vectordb_handler import VectorDBManagerOpenAI
vdb = VectorDBManagerOpenAI(
        db_path="./chroma_db",
        collection_name="heart_data_vb"
    )

In [9]:
text = [chunk.page_content for chunk in chunks]
metadata = [chunk.metadata for chunk in chunks]

In [10]:
stored_ids = vdb.store_documents(
        text = text,
        metadatas = metadata
    )

✅ Stored 261 document(s) in vector DB 'heart_data_vb'


In [12]:
vdb.peek(160)

{'ids': ['da57e359-5e35-405f-8b81-c494cd4f69c9',
  'fc39bd7b-b347-412d-b355-1174ba10c498',
  '400002ea-de03-445a-b842-010982db9894',
  'b3c7d58d-4d39-4cb1-9cf0-941a30a9cc3a',
  '64902cfd-db1d-40f0-8b36-8aef5f950a6a',
  '4d3edc88-f046-40c6-a1d0-eb8c492566e3',
  'bdb6cc62-7bd8-4abe-8dcf-fa8ee86547fe',
  'ea33b91b-4e20-47db-9847-c03bf845c750',
  '5eec07d0-eada-44b6-aece-fc9dbef17714',
  '0f3e3b26-eb6c-4c61-b2a2-f362fa1740b8',
  'b4d48289-7d66-4920-83e8-592bdda225c6',
  'd1cbfabd-932e-40f2-bbcb-109fcc197f33',
  '1488d7c1-8aef-4f64-8572-b94466884bc5',
  '59addd80-870a-4ab2-87d4-dd3d5f7eb9ef',
  '1aeb031d-8565-41c6-97bd-3124aa87c33d',
  'eedc5ce4-79e3-49b1-8210-11f4291873f6',
  '645c67a3-407b-4ebe-b9bf-4be2f3799257',
  '62565f3c-3356-4319-8760-ee7573ae89c1',
  '464f7c4d-51f7-452b-803a-342d3c7bd6c1',
  '7cb94e8b-8485-4296-92aa-1eddd261e06e',
  'dc5c0287-0536-4264-b8a9-70a5c44339f4',
  '1fe23d09-0932-418d-b558-587b21fb8fad',
  '00100da2-a70a-49b6-bed0-2a7091552c1a',
  '68f85a50-b801-414d-83c9-

QUERRYING DB

In [13]:
x = vdb.query(query= "Unusual thickening, enlargement or stiffening of your heart muscle, What is this condition calle?")

In [16]:
x["documents"]

[['Conditions and Disorders \nWhat are the common conditions and disorders that affect your \nheart? \nHeart conditions are among the most common types of disorders. In the United States, heart \ndisease is the leading cause of death. \nCommon conditions that affect your heart include: \n• \nArrhythmia: A heartbeat that’s too fast, too slow or beats with an irregular rhythm. \n• \nCardiomyopathy: Unusual thickening, enlargement or stiffening of your heart \nmuscle. \n• \nCongestive heart failure: Your heart is too stiff or too weak to properly pump blood \nthroughout your body. \n• \nCoronary artery disease: Plaque buildup that leads to narrow coronary arteries. \n• \nDiabetes: Your blood sugar is higher than it should be. \n• \nHeart attack (myocardial infarction): A sudden coronary artery blockage that cuts off \noxygen to part of your heart muscle. \n• \nHeart valve disease: A valve in your heart isn’t working right. \n• \nHigh blood pressure: Your blood is pushing too hard against 

LLM Response

In [9]:
import google.genai as genai
import os
import dotenv

dotenv.load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key = GOOGLE_API_KEY)

# 2️⃣ Define the function
def llm(query, passage):
    prompt = f"Context:\n{passage}\n\nQuestion: {query}"
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
    )
    print(response.text)


In [17]:
querry ="Symptoms of heart conditions give me top 5"

llm(passage= x, query=querry)

NameError: name 'llm' is not defined

In [5]:
from src.backend.llm import llm
llm_obj = llm()


In [7]:
response = llm_obj.genrate_response(query= querry, context= x)

In [9]:
print(response.text)

Symptoms of heart conditions include:
*   Chest pain
*   Heart palpitations
*   Dizziness
*   Shortness of breath
*   Fatigue
*   Swelling in your lower body


Creating conversation history

In [1]:
%cd "C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag"

C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag


In [4]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain

# Initialize memory (keeps last 10 messages)
memory = ConversationBufferWindowMemory(
    k=10,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# Create conversational chain
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vdb.as_retriever(),
    memory=memory
)

# Use it
# response = qa_chain({"question": user_query})

ModuleNotFoundError: No module named 'langchain.memory'

In [5]:
from collections import deque

class ConversationHistory:
    def __init__(self, max_history=10):
        self.history = deque(maxlen=max_history)
    
    def add_exchange(self, user_query, assistant_response):
        self.history.append({
            "user": user_query,
            "assistant": assistant_response
        })
    
    def get_history_string(self):
        """Format history for LLM context"""
        history_text = ""
        for exchange in self.history:
            history_text += f"User: {exchange['user']}\nAssistant: {exchange['assistant']}\n\n"
        return history_text
    
    def clear(self):
        self.history.clear()

In [8]:
def llm_with_history(client,query, passage, conversation_history):
    # Get formatted history
    history_context = conversation_history.get_history_string()
    
    # Build prompt with history
    prompt = f"""Previous Conversation:
    {history_context}

    Context (from documents):
    {passage}

    Current Question: {query}

    Please answer the current question, considering the conversation history if relevant."""
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
    )
    
    # Add this exchange to history
    conversation_history.add_exchange(query, response.text)
    
    return response

In [11]:
from src.constants import GOOGLE_API_KEY
from google import genai as genai
# Initialize conversation history
chat_history = ConversationHistory(max_history=10)

from src.backend.vectordb_handler import VectorDBManagerOpenAI
vdb = VectorDBManagerOpenAI(
        db_path="./chroma_db",
        collection_name="heart_data_vb"
)

# Chat loop
client = genai.Client(api_key = GOOGLE_API_KEY)
while True:
    user_query = input("You: ")
    
    if user_query.lower() in ['exit', 'quit']:
        break
    
    # Query vector DB for relevant context
    relevant_docs = vdb.query(query=user_query)
    
    # Generate response with history
    response = llm_with_history(client,user_query, relevant_docs, chat_history)
    
    print(f"Assistant: {response.text}\n")

KeyboardInterrupt: 